In [29]:
# Inherited: `data` 
#%run "C:\Users\Asus\Desktop\Coding\API_Flask\side_BenckMark\embedding_Workspace\bench\2_Preprocess.ipynb" 

import dill
# Load session 
dill.load_session('2_Proprocess.pkl') 

In [30]:
data.head(2)

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding,review_Embedding,semantic_Label,label_Embedding
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.007018072064965963, -0.02731654793024063, 0...","[0.007018072064965963, -0.02731654793024063, 0...",positive,"[-0.006393480114638805, -0.008763186633586884,..."
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.003140551969408989, -0.009995664469897747,...","[-0.003140551969408989, -0.009995664469897747,...",positive,"[-0.006393480114638805, -0.008763186633586884,..."


### `Predictions`

---
### `Main Calculations` 

#### `Setup` 

##### `Classification Report` 
* methods: "`create_Report`", "`extract_summary_metrics`" 

In [31]:
from sklearn.metrics import classification_report 

In [32]:
def create_Report(current_Prediction):
    global cosine_Report 
    global title_List, precision_List, recall_List, f1_List, support_List
    
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    title_List, precision_List, recall_List, f1_List, support_List = [], [], [], [], [] 

    for line in report.split("\n")[2:-5]:  # Skip the first two lines and the last five lines of the report
        # Split the line by spaces but exclude empty splits
        values = [value for value in line.split("  ") if value]
        
        # Extract title
        try:
            title = values[0].strip()
        except:
            title = float('NaN')
        title_List.append(title)
        
        # Extract precision
        try:
            precision = float(values[1])
        except:
            precision = float('NaN')
        precision_List.append(precision)
        
        # Extract recall
        try:
            recall = float(values[2])
        except:
            recall = float('NaN')
        recall_List.append(recall)
        
        # Extract f1-score
        try:
            f1 = float(values[3])
        except:
            f1 = float('NaN')
        f1_List.append(f1)
        
        # Extract support
        try:
            support = int(values[4])
        except:
            support = float('NaN')
        support_List.append(support)

    # Create dataframe from the values 
    cosine_Report = pd.DataFrame({
        'title': title_List, 
        'precision': precision_List, 
        'recall': recall_List, 
        'f1': f1_List, 
        'support': support_List
    })    
    
    return cosine_Report


In [33]:
def extract_summary_metrics(current_Prediction):
    # Get the classification report with zero_division=1 to handle division by zero
    report = classification_report(data["semantic_Label"], data[current_Prediction], zero_division=1)
    
    # Split the report by lines
    lines = report.split("\n")
    
    # Extract accuracy
    try:
        accuracy = float(lines[-2].split(":")[1].strip())
    except:
        accuracy = float('NaN')
    
    # Extract macro avg metrics
    macro_avg = lines[-4].split()
    try:
        macro_precision = float(macro_avg[2])
    except:
        macro_precision = float('NaN')
    try:
        macro_recall = float(macro_avg[3])
    except:
        macro_recall = float('NaN')
    try:
        macro_f1 = float(macro_avg[4])
    except:
        macro_f1 = float('NaN')
    try:
        macro_support = int(macro_avg[5])
    except:
        macro_support = float('NaN')
    
    # Extract weighted avg metrics
    weighted_avg = lines[-3].split()
    try:
        weighted_precision = float(weighted_avg[2])
    except:
        weighted_precision = float('NaN')
    try:
        weighted_recall = float(weighted_avg[3])
    except:
        weighted_recall = float('NaN')
    try:
        weighted_f1 = float(weighted_avg[4])
    except:
        weighted_f1 = float('NaN')
    try:
        weighted_support = int(weighted_avg[5])
    except:
        weighted_support = float('NaN')
    
    # Create a DataFrame with the extracted metrics
    summary_df = pd.DataFrame({
        'title': ['accuracy', 'macro avg', 'weighted avg'],
        'precision': [float('NaN'), macro_precision, weighted_precision],
        'recall': [float('NaN'), macro_recall, weighted_recall],
        'f1': [float('NaN'), macro_f1, weighted_f1],
        'support': [accuracy, macro_support, weighted_support]
    })
    
    return summary_df


##### `Similarity Calculations`
* method: "`label_Similarity_Score`", "`predict_1`", "`predict_2`"

In [34]:
from scipy.spatial import distance

def label_Similarity_Score(review_embedding, label_embeddings, method='Cosine'):
    scores = {}
    
    for label, embedding in label_embeddings.items():
        if method == 'Cosine':
            scores[label] = 1 - distance.cosine(review_embedding, embedding)
        elif method == 'Euclidean':
            scores[label] = 1 / (1 + distance.euclidean(review_embedding, embedding))
        elif method == 'Manhattan':
            scores[label] = 1 / (1 + distance.cityblock(review_embedding, embedding))
        elif method == 'Minkowski':
            scores[label] = 1 / (1 + distance.minkowski(review_embedding, embedding, p=3))
        elif method == 'Pearson':
            scores[label] = np.corrcoef(review_embedding, embedding)[0, 1] # Pearson is not a distance metric, so we use correlation coefficient instead 
        elif method == 'Canberra':
            scores[label] = 1 / (1 + distance.canberra(review_embedding, embedding))
       
        else:
            raise ValueError(f"Unknown method: {method}")
    
    return scores

def predict_1(similarity_scores, n=1):
    # Sort labels based on scores and get the top n labels
    sorted_labels = sorted(similarity_scores, key=similarity_scores.get, reverse=True)
    return sorted_labels[:n][0]

def predict_2(scores):
    max_index = np.argmax(scores)
    if max_index < len(current_Labels):
        return current_Labels[max_index]
    else:
        return 'External'  # or any other placeholder 

#### `Process` 

##### `Similarities & Predictions` 
* Output: "`Probabilities`" & "`Labels`" 

In [35]:
# Get Similarity_Score for each review
data["Similarity_Score_Cosine"]    = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Cosine')) 
data["Similarity_Score_Euclidean"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Euclidean')) 
data["Similarity_Score_Manhattan"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Manhattan')) 
data["Similarity_Score_Minkowski"] = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Minkowski')) 
data["Similarity_Score_Pearson"]   = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Pearson')) 
data["Similarity_Score_Canberra"]  = data["review_Embedding"].apply(lambda x: label_Similarity_Score(x, label_Dict, method='Canberra')) 

# Assign top 2 labels for each review 
data["predicted_labels_Cosine"]    = data["Similarity_Score_Cosine"].    apply(lambda x: predict_1(x))
data["predicted_labels_Euclidean"] = data["Similarity_Score_Euclidean"]. apply(lambda x: predict_1(x))
data["predicted_labels_Manhattan"] = data["Similarity_Score_Manhattan"]. apply(lambda x: predict_1(x))
data["predicted_labels_Minkowski"] = data["Similarity_Score_Minkowski"]. apply(lambda x: predict_1(x))
data["predicted_labels_Pearson"]   = data["Similarity_Score_Pearson"].   apply(lambda x: predict_1(x))
data["predicted_labels_Canberra"]  = data["Similarity_Score_Canberra"].  apply(lambda x: predict_1(x))

---
### `Checkpoint` 

In [36]:
import dill

# Save the entire session
dill.dump_session('3_Process.pkl')  
# Actialize the session
#dill.load_session('session.pkl') 